In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

# Connect to Weaviate
client = weaviate.connect_to_weaviate_cloud(
	cluster_url=CLUSTER_URL,
	auth_credentials=weaviate.auth.AuthApiKey(API_KEY),
	headers={
		"X-OpenAI-Api-Key": OPENAI_API_KEY,
		"X-Cohere-Api-Key": COHERE_API_KEY
	},
	additional_config=AdditionalConfig(
		timeout=Timeout(init=30, query=60, insert=120)
	)
)

ready = client.is_ready()
server_version = client.get_meta()["version"]
client_version = weaviate.__version__
live = client.is_live()
connected = client.is_connected()

print(f"Weaviate Ready: {ready}")
print(f"Weaviate Client Version: {client_version}")
print(f"Weaviate Server Version: {server_version}")
print(f"Weaviate Live: {client.is_live()}")
print(f"Client Connected: {connected}")


In [ ]:
# Delete with condition
from weaviate.classes.query import Filter

collection = client.collections.get("<COLLECTION_NAME>")
to_remove = collection.query.fetch_objects(limit=5)
print(to_remove)

result=collection.data.delete_many(
    where=Filter.by_property("<PROPERTY_NAME>").contains_any(to_remove[:5])
)
print(result)

In [ ]:
# Delete all objects within a tenant
multi_collection = client.collections.get("MultiTenancyCollection")
response = multi_collection.with_tenant("your_tenant_name").data.delete_many(
    where=Filter.by_property("_id").contains_any("*")
)
# OR
# multi_collection = client.collections.get("MultiTenancyCollection")
# multi_collection.tenants.remove(["your_tenant_name"])

In [ ]:
# # Delete a single collection
client.collections.delete("<COLLECTION_NAME>") # THIS WILL DELETE THE SPECIFIED COLLECTION AND ALL ITS OBJECTS

# # Delete multiple collections
client.collections.delete(["<COLLECTION_NAME>", "<COLLECTION_NAME>"])

# Delete all collections in the Weaviate instance
client.collections.delete_all()